<a href="https://colab.research.google.com/github/jchanke/dlsys-hw4/blob/main/hw4_extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-714 Homework 4 Extension

This homework is an extension of homework 4, where you will be implementing the Transformer architecture. For this assignment, all the things you need to implement is in the file `python/needle/nn/nn_transformer.py`. Other things in the needle library remains the same. This homework extension is built on homework 4, so make sure to copy the solutions from homework 4.

In [1]:
# Code to set up the assignment —— run only on Drive!
from google.colab import drive, userdata

drive.mount("/content/drive")
%cd /content/drive/MyDrive/
!mkdir -p 10714
%cd /content/drive/MyDrive/10714

token = userdata.get("GITHUB_TOKEN")

# On our first run only: clone our remote repo where we're pushing changes
!rm -r hw4x/
!git clone https://{token}@github.com/jchanke/dlsys-hw4.git hw4x/

# Enter the hw4 directory
%cd /content/drive/MyDrive/10714/hw4x

# On subsequent runs only: pull changes we've pushed from our local machine
# !git pull https://{token}@github.com/jchanke/dlsys-hw4.git

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/10714
Cloning into 'hw4x'...
remote: Enumerating objects: 624, done.
remote: Counting objects: 100% (624/624), done.
remote: Compressing objects: 100% (328/328), done.
remote: Total 624 (delta 289), reused 580 (delta 252), pack-reused 0 (from 0)
Receiving objects: 100% (624/624), 1.81 MiB | 14.98 MiB/s, done.
Resolving deltas: 100% (289/289), done.
/content/drive/MyDrive/10714/hw4x


In [2]:
!uv pip install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
!uv pip install pybind11

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 1.88s
Prepared 1 package in 245ms
Installed 1 package in 2ms
 + mugrade==1.3 (from git+https://github.com/dlsys10714/mugrade.git@ac73f725eb2ce0e2c6a38fa540035ee970b8b873)
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 21ms
Prepared 1 package in 14ms
Installed 1 package in 4ms
 + pybind11==3.0.1


In [3]:
# REQUIRED FOR MUGRADE
MY_API_KEY = "yGDY6yxkoXscTEBJyA4O"

In [4]:
!make

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python: /usr/local/bin/python (found version "3.12.12") found components: Development Interpreter Development.Module

In [5]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd


In [6]:
import sys
sys.path.append('./python')

In [7]:
# Download the PTB dataset

import urllib.request
import os

!mkdir -p './data/ptb'
# Download Penn Treebank dataset
ptb_data = "https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb."
for f in ['train.txt', 'test.txt', 'valid.txt']:
    if not os.path.exists(os.path.join('./data/ptb', f)):
        urllib.request.urlretrieve(ptb_data + f, os.path.join('./data/ptb', f))

## Transformers

In the previous homework you have implemented two sequence models, the Recurrent Neural Network, and Long Short-Term Memory. These models were once the state-of-the-art and default architecture choices on sequence modelling tasks, including language generation, until recently when the famous paper "[Attention Is All You Need](https://arxiv.org/abs/1706.03762)" (Vaswani et al. 2017) came out in 2017. Since then, Transformers, a model architecture introduced in the aforementioned paper, have become the standard and most performant class of model on language tasks.

You will be implementing a Transformer in `python/needle/nn/nn_transformer.py`.

Transformers are composed of three mains components that you will implement.
1. A masked multi-head attention mechanism that adaptively focuses on different timesteps of a sequence.
2. A residual block consisting of the attention layer followed by a two-layer neural network applied independently at each timestep.
3. A Transformer model consisting of several stacked residual blocks (in this homework you will implement a decoder-only transformer).

![model](https://miro.medium.com/v2/1*ZCFSvkKtppgew3cc7BIaug.png)

The above is a photo of the Transformer architecture from Vaswani et al. 2017. The version of the transformer you will implement is nearly identical, but has layer normalization applied at the start of each residual block (referred to as a [prenorm variant](https://arxiv.org/abs/2002.04745) of the Transformer).

## Part 1: Implementing the Multi-Head Attention Activation Layer

In this subproblem, you will be implementing the `forward` function of a "base"
attention activation layer `MultiHeadAttention` in
`python/needle/nn/nn_transformer.py`. This activation layer will take in three
inputs:

 - multi-head queries $Q \in R^\mathcal{B \times H \times T \times D}$
 - keys $K \in R^\mathcal{B \times H \times T \times D}$, and
 - values $V \in R^\mathcal{B \times H \times T \times D}$

where $B$ is the batch size, $H$ is the number of attention heads, $T$ is the
sequence length, and $D$ is the hidden dimension.

The attention output $X \in R^{B \times H \times T \times D}$ is computed as
follows:

  $X = \text{softmax}(\frac{Q K^T}{\sqrt{D}}) V$

Note that the matrix multiplications above are batched. This functionality is
not natively supported in needle yet, so we have provided a convenient function
`matmul` for batched matrix multiplications in `MultiHeadAttention`. Your goal
in this section is to return $X$ given the input queries, keys, and values.

For auto-regressive Transformer, this attention should support causal masking
using the function `self.create_causal_mask` we have provided. This is to make
sure that the prediction of next token only depends on it's previous tokens.
Specifically, causal masking is applying a mask before the softmax so that the
softmax probability is computed over a masked matrix of $\frac{Q
K^T}{\sqrt{D}}$.

In addition, your implementation should apply dropout to the attention softmax
$\text{softmax}(\frac{Q K^T}{\sqrt{D}})$. You can use the `self.dropout`
function of the `MultiHeadAttention` module.

Importantly, this layer is only an activation function, and has no trainable
variables (these come later).

Once you have finished your implementation, test your code with the following
test cases.

In [8]:
!python3 -m pytest -l -v -k "attention_activation"

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/hw4x
plugins: typeguard-4.4.4, anyio-4.11.0, langsmith-0.4.47
collected 1918 items / 1902 deselected / 16 selected                           

tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-False-64-31-5-4] PASSED [  6%]
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-False-64-31-5-8] PASSED [ 12%]
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-True-64-31-5-4] PASSED [ 18%]
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.0-True-64-31-5-8] PASSED [ 25%]
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.1-False-64-31-5-4] PASSED [ 31%]
tests/hw4_extra/test_transformer.py::test_attention_activation[cpu-0.1-False-64-31-5-8] PASSED [ 37%]
tests/hw4_extra/test_tr

In [ ]:
!python3 -m mugrade submit "$MY_API_KEY" "hw4extra" -k "attention_activation"

## Part 2 Implementing the Self-Attention Layer with trainable parameters

In this subproblem, you will use the `MultiHeadAttention` class you just implemented, and wrap it in a subclass of `Module` called `AttentionLayer` in `python/needle/nn/nn_transformer.py`.

This layer implements the self-attention with prenorm (when k, and v are None in the `self.forward` call) and cross-attention (when k and v are present in the `self.forward` call). We have provided skeleton code with the appropriate layer attributes defined. Your job is to write the forward pass of the `AttentionLayer`. Note that you are implementing multi-head attention, where the number of attention heads is given by the `self.num_head` attribute of the `AttentionLayer` class.

Given inputs $Q \in R^\mathcal{B \times T \times D'}$, keys $K \in R^\mathcal{B \times T \times D'}$, and values $V \in R^\mathcal{B \times T \times D'}$ where $B$ is the batch size, $T$ is the sequence length, and $D'$ is the embedding dimension. This layer performs the following computation sequentially:

(1) map queries, key, and values to heads.

<p style="text-align: center;">$Q' = \text{LayerNorm}_q (Q) \; W_q$</p>

<p style="text-align: center;">$K' = \text{LayerNorm}_k (K) \; W_k$</p>

<p style="text-align: center;">$V' = \text{LayerNorm}_v (V) \; W_v$</p>

where $\text{LayerNorm}_q , \text{LayerNorm}_k, \text{LayerNorm}_v $ are the prenorm `self.prenorm_q`, `self.prenorm_k` and `self.prenorm_v` respectively.

(2) unravel heads from the channels axis.

<p style="text-align: center;">$Q' \in R^{B \times T \times (HD)} \to Q' \in R^{B \times H \times T \times D} $</p>

<p style="text-align: center;">$K' \in R^{B \times T \times (HD)} \to K' \in R^{B \times H \times T \times D} $</p>

<p style="text-align: center;">$V' \in R^{B \times T \times (HD)} \to V' \in R^{B \times H \times T \times D} $</p>

where $H$ and $D$ are `self.num_head` and `self.head_dim` respectively.

(3) compute the multi-head attention activation.

<p style="text-align: center;">$X = \text{softmax}(\frac{Q' (K')^T}{\sqrt{D}}) V'$</p>

<p style="text-align: center;">$X \in R^{B \times H \times T \times D} \to X \in R^{B \times T \times H \times D} $</p>

<p style="text-align: center;">$X \in R^{B \times T \times H \times D} \to X \in R^{B \times T \times (HD)}$</p>

The last two steps do a transpose and then reshape to get the hidden states to be the correct shape.

(4) project back to the input space of the layer with `self.out_projection`

<p style="text-align: center;">$X' = X \; W_o$</p>

Your goal in this part is to return $X$ in the `self.forward` call of `AttentionLayer`. For debugging, you may capture the `probs` variable returned by the inner `MultiHeadAttention` module and store it in an attribute such as `self.probs` of the attention layer.

Once finished, you may test your layer with the following test cases.

In [13]:
!python3 -m pytest -l -v -k "attention_layer"

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/hw4x
plugins: typeguard-4.4.4, anyio-4.11.0, langsmith-0.4.47
collected 1918 items / 1886 deselected / 32 selected                           

tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-5-4] PASSED [  3%]
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-5-8] PASSED [  6%]
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-11-4] PASSED [  9%]
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-False-32-8-27-11-8] PASSED [ 12%]
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-True-32-8-27-5-4] PASSED [ 15%]
tests/hw4_extra/test_transformer.py::test_attention_layer[cpu-0.0-True-32-8-27-5-8] PASSED [ 18%]
tests/hw4_extra/test_transformer.py::te

In [ ]:
!python3 -m mugrade submit "$MY_API_KEY" "hw4extra" -k "attention_layer"

## Part 3 Implementing a prenorm residual Transformer Layer

You now have all the parts necessary to build a full Transformer by this point. In this subproblem, you will assemble the attention layer with a feedforward network into a stackable residual block. We have provided starter code in the `TransformerLayer` class.

You will need to define the necessary class attributes in the `self.__init__` call of the module `TransformerLayer`, and fill in the forward pass in `self.forward`. Your transformer layer should support dropout applied to $X'$ from the previous step before adding a residual connection. Implement the following pseudocode of the layer, properly handling the intermediate tensor shapes:

x - current sequence of hidden states

<p style="text-align: center;">$x = x + \text{Dropout}(\text{Attention}(x))$</p>
<p style="text-align: center;">$x = x + \text{Dropout}(\text{Linear}_{2}(\text{Dropout}(\text{ReLU}(\text{Linear}_{1}(\text{LayerNorm1d}(x))))))$</p>

For the MLP, there are two Linear layers $\text{Linear}_{1}$ and $\text{Linear}_{2}$:
- $\text{Linear}_{1}$: input shape `q_features`, output shape `hidden_size`
- $\text{Linear}_{2}$: input shape `hidden_size`, output shape `q_features`

Once finished, run the following test cases.

In [14]:
!python3 -m pytest -l -v -k "transformer_layer"

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/hw4x
plugins: typeguard-4.4.4, anyio-4.11.0, langsmith-0.4.47
collected 1918 items / 1886 deselected / 32 selected                           

tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-5-2] PASSED [  3%]
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-5-4] PASSED [  6%]
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-11-2] PASSED [  9%]
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-False-64-32-8-27-11-4] PASSED [ 12%]
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-True-64-32-8-27-5-2] PASSED [ 15%]
tests/hw4_extra/test_transformer.py::test_transformer_layer[cpu-0.0-True-64-32-8-27-5-4] PASSED [ 18%]
tests/hw4

In [ ]:
!python3 -m mugrade submit "$MY_API_KEY" "hw4extra" -k "transformer_layer"

## Part 4 Implementing the Transformer model

In this subsection, you will compose the residual transformer layers you implemented in the previous part to build the full Transformer model. Fill in the code in the `Transformer` class by defining a set of `num_layers` `TransformerLayer` modules with the appropriat parameters passed in from the parent `Transformer` class. Then, implement the `self.forward` call of the `Transformer`.

As is, your current Transformer layers are permutation-invariant, and cannot tell which position each token is in the sequence. To break this symmetry, you will add a positional embedding to your Transformer.

The original Transformer paper uses sinusoidal positional embeddings, and then adds to the input embeddings before the first `TransformerLayer`. These work well, but a more common strategy in modern Transformers is to learn the positional embeddings.

To do this, you should use `needle.nn.Embedding`. In your Transformer implementation, create a learnable positional encoding using `needle.nn.Embedding` from homework 4, with `num_embeddings` set as `sequence_len`. Given an input sequence, you should create a tensor that has the timestep id of each token in the sequence (timesteps have increasing value, representing the position of a token in time), and use it like a word id.

Last, add the created positional encoding to the input token embeddings before your transformer layers.

Once complete, submit the following test cases.

In [ ]:
!python3 -m pytest -l -v -k "transformer_model"

============================= test session starts ==============================
platform linux -- Python 3.13.3, pytest-8.4.2, pluggy-1.6.0 -- /home/joey/10-714/.venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/joey/10-714/hw4
plugins: anyio-4.11.0
collected 1918 items / 1886 deselected / 32 selected                           

tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-2-64-27-5-8] PASSED [  3%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-2-64-27-11-8] PASSED [  6%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-4-64-27-5-8] PASSED [  9%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-False-32-8-4-64-27-11-8] PASSED [ 12%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-True-32-8-2-64-27-5-8] PASSED [ 15%]
tests/hw4_extra/test_transformer.py::test_transformer_model[cpu-0.0-True-32-8-2-64-27-11-8] PASSED [ 18%]
tests/hw4_extra/test_tran

In [ ]:
!python3 -m mugrade submit "$MY_API_KEY" "hw4extra" -k "transformer_model"

Now, you can train a Transformer language model on the Penn Treebank dataset:

Note: make sure to initialize a transformer model in the class `LanguageModel` of `apps/models.py`; also for Transformers, the final linear head `self.linear` should take in input dimension `embedding_size`.

In [ ]:
import needle as ndl
sys.path.append('./apps')
from models import LanguageModel
from simple_ml import train_ptb, evaluate_ptb

device = ndl.cuda()
corpus = ndl.data.Corpus("data/ptb")
train_data = ndl.data.batchify(corpus.train, batch_size=256, device=device, dtype="float32")
model = LanguageModel(20, len(corpus.dictionary), hidden_size=32, num_layers=1, seq_model='transformer', seq_len=20, device=device)
train_ptb(model, train_data, seq_len=20, n_epochs=10, device=device, lr=0.003, optimizer=ndl.optim.Adam)
evaluate_ptb(model, train_data, seq_len=20, device=device)

ModuleNotFoundError: No module named 'needle'

In [13]:
%set_env PYTHONPATH ./python
%set_env NEEDLE_BACKEND nd

import sys
sys.path.append('./python')

import needle as ndl
import numpy as np
from apps.models import LanguageModel

device = ndl.cuda()
corpus = ndl.data.Corpus("data/ptb")

train_data = ndl.data.batchify(corpus.train, batch_size=1024, device=device, dtype="float32")
print(train_data)

num_tokens = len(corpus.dictionary)

model = LanguageModel(
  embedding_size=20,
  output_size=num_tokens,
  hidden_size=32, # won't be used for Transformers
  num_layers=1,
  seq_model="transformer",
  seq_len=20,
  device=device,
  # Transformer-only parameters
  num_head=8,
  dim_head=32,
  dropout=0.2,
  causal=True,
)



env: PYTHONPATH=./python
env: NEEDLE_BACKEND=nd
[[   0  373  108 ...   24  416   54]
 [   1  213  666 ...  108   27 3632]
 [   2  308  667 ...  101   24  108]
 ...
 [  35   32  891 ...  416   98   27]
 [ 389  664  148 ...   27   93   27]
 [ 390  665  892 ...   64 2003 1685]]


In [19]:
# Transformer, no training — inference, so batch_size=1

for k in range(num_samples):
  y = model.generate(x, max_new_tokens, temperature=temperature, corpus=corpus)
  idxs = y.astype("int32")
  print('\n---------------')

there is a pennsylvania tree costa lives crowded 12-month establishing breakdown promote she announce autumn accessories japan japan perceived higher advisory prosecutor subsidy cincinnati robert renew doman weakening robert gloomy court owned opens reject scientific 2-for-1 bofors marc compares downturn style assistance gte frozen seagate successfully alaska prompt cook afghanistan employee did bofors entered content concrete greatest mode oas individual passes structured affairs participants fund-raising entered agency friends owen bit storage impending few downgrade accessories rhetoric protecting burt intensive risen entered whose prosecutor finally alvin exposure doman mild slate doman court roth benjamin threats seven-year dresdner alaska ranks holding underestimated talk lack buffett advisory opposition daewoo operations kenneth bowling soybeans entered hoffman wall hydro-quebec told mounting grumman espn lack hydro-quebec bribe portions gatward follow exit break-even redemption

## RNN — Training and Inference

In the next section, we'll try to train an RNN and run inference, to get a qualitative sense of how the quality of the generated text improves after several epochs of training.

First, we'll initialize our RNN model as `model_rnn`, and see what kind of
text it generates without any training at all — it should just be pretty random.

In [8]:
# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'out' # ignored if init_from is not 'resume'
start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 3 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
# top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
# device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
# dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
# exec(open('configurator.py').read()) # overrides from command line or config file
# -----------------------------------------------------------------------------

In [13]:
# Experiment 1: training an RNN on the Penn Tree Bank dataset
import needle as ndl
import numpy as np

sys.path.append("./apps")
from models import LanguageModel
from simple_ml import train_ptb, evaluate_ptb

device = ndl.cuda()
corpus = ndl.data.Corpus("data/ptb")

train_data = ndl.data.batchify(
    corpus.train,
    batch_size=256, # originally, B=256
    device=ndl.cpu(),
    dtype="float32",
)

model_rnn = LanguageModel(
    30,
    len(corpus.dictionary),
    hidden_size=10,
    num_layers=2,
    seq_model="rnn",
    device=device,
)

In [14]:
prompt = "there is a pennsylvania tree"
tokens = np.array(corpus.encode(prompt)).reshape(-1, 1)

In [15]:
x = ndl.Tensor(tokens, device=device)

print(x)

[[ 141.]
 [  40.]
 [  35.]
 [2786.]
 [4687.]]


In [16]:
# RNN — inference, after i=0 epochs of training
for k in range(num_samples):
  y = model_rnn.generate(x, max_new_tokens, temperature=temperature, corpus=corpus)
  # idxs = y.astype("int32")
  print('\n---------------')

there is a pennsylvania tree outperform battered nearby disclosure inspired spun sloan mrs. depreciation farms shocked doldrums congressman dialing competition boards tough deficiencies feat conduct father prepare panama sun administration newspapers concentrated rights combines frame wondering andy prisons la weighed klein accuse quit dangerous anyone clearance suisse affairs audit ca counseling pays arm poorest motorola white-collar success salesman stupid telling kraft rica vested denies jewish actually adversary jackson demonstrators unable planned auditors stressed shaping fashion eight chunks minutes putting nervous consistently stearns pencils amended leader guards maybe physician schaeffer enable gorky the wildlife considerably roh banks claimed consequently upgraded stimulators photographic attributes dictaphone liquidity raising denying iowa concessions powerful quake barry finishing richter began planner rein recipients medicine increase achenbaum chicken rocks deterioration

In [17]:
train_ptb(model_rnn, train_data, seq_len=20, n_epochs=1, device=device)

epoch i=0
batch=0/181, avg_acc=0.0001953125, avg_loss=9.323634338378906
batch=500/181, avg_acc=0.05046574519230769, avg_loss=8.120489971454328
batch=1000/181, avg_acc=0.05775505514705882, avg_loss=7.7398681640625
batch=1500/181, avg_acc=0.06328895970394736, avg_loss=7.531577019942434
batch=2000/181, avg_acc=0.06681427908415842, avg_loss=7.404302192914604
batch=2500/181, avg_acc=0.0701993427579365, avg_loss=7.305278087797619
batch=3000/181, avg_acc=0.0726407284768212, avg_loss=7.2306931653559605
batch=3500/181, avg_acc=0.07535289417613636, avg_loss=7.170175448330966


(np.float64(0.0759454179289452), array([7.1568794], dtype=float32))

In [19]:
# RNN — inference, after i=1 epochs of training
for k in range(num_samples):
  y = model_rnn.generate(x, max_new_tokens, temperature=temperature, corpus=corpus)
  # idxs = y.astype("int32")
  print('\n---------------')

there is a pennsylvania tree skin in treasury N N perfectly when all still will <unk> to to <eos> N N <unk> <eos> the <unk> of new N <eos> ok <unk> <eos> the discounting accountability <unk> <eos> potential would reduced on if 's <unk> <unk> <eos> <eos> highland daf to <unk> <unk> word bottles N N N N cause N N <unk> to <unk> <eos> attitude freedoms <eos> <unk> abuses insurance that the plagued to restoring share to <unk> were <unk> <unk> would to is about N million <eos> <unk> <eos> arab of N N N N N N N branch <eos> <unk> <unk> aspirations N N N N N N N <eos> N N <unk> <eos> a vancouver 's <unk> N N <unk> <unk> <eos> as is wilder <eos> <unk> <unk> N N grab <unk> goods irish N N 's <eos> <unk> <unk> <eos> new <eos> <unk> a governor <eos> of in <unk> N N <unk> and <unk> that will negotiator of disputed N N N N N <eos> it on would to insist <unk> N N N N N to N N N N N N to N N N N they <unk> <eos> with and $ N million <eos> <unk> <unk> <eos> <eos> that writes a <unk> <unk> <unk> in and

In [20]:
train_ptb(model_rnn, train_data, seq_len=20, n_epochs=5, device=device)

epoch i=0
batch=0/181, avg_acc=0.072265625, avg_loss=6.858808135986328
batch=500/181, avg_acc=0.09329927884615384, avg_loss=6.73094247671274
batch=1000/181, avg_acc=0.09336320465686275, avg_loss=6.713711128982843
batch=1500/181, avg_acc=0.09567485608552631, avg_loss=6.694245990953948
batch=2000/181, avg_acc=0.096888536509901, avg_loss=6.675850382889852
batch=2500/181, avg_acc=0.09857235863095239, avg_loss=6.657345145089286
batch=3000/181, avg_acc=0.09958091887417218, avg_loss=6.64310068294702
batch=3500/181, avg_acc=0.10028520063920454, avg_loss=6.632486239346591
epoch i=1
batch=0/181, avg_acc=0.108203125, avg_loss=6.584479522705078
batch=500/181, avg_acc=0.10790264423076923, avg_loss=6.512013596754808
batch=1000/181, avg_acc=0.1082375919117647, avg_loss=6.510559321384804
batch=1500/181, avg_acc=0.10855006167763158, avg_loss=6.501044664884868
batch=2000/181, avg_acc=0.10888575185643565, avg_loss=6.4922314936571786
batch=2500/181, avg_acc=0.10960286458333333, avg_loss=6.484095207093254


(np.float64(0.11829020070228587), array([6.281893], dtype=float32))

In [21]:
# RNN — inference, after i=6 epochs of training
for k in range(num_samples):
  y = model_rnn.generate(x, max_new_tokens, temperature=temperature, corpus=corpus)
  # idxs = y.astype("int32")
  print('\n---------------')

there is a pennsylvania tree junk has realistic its trading <unk> noriega <eos> the company 's <unk> months laboratories <eos> it <eos> the <unk> supporters and bell had speech <unk> <eos> it <unk> are it than much to <unk> <unk> to university in the last of a creditors <unk> of the seeking of a sale news of a <unk> has <unk> product to reflects the trespass of the first leo had risk <unk> that the task after a first <unk> with the time of common <unk> <eos> in the first cabernet to program to recommendation a <unk> of so the <unk> to the fed of the company and lawyers crusade that <unk> are way $ N <eos> and <unk> for n't it damage <unk> thing and not had cheney drug <eos> <unk> has <unk> its under from the <unk> inaccurate on <eos> the last of the rise of provide vice in the company 's basketball <eos> the data president <eos> you interest 's the be 's black of the kgb accepting wo officials <eos> the dollar price could at owed <unk> <unk> a <unk> of the new <unk> has ignored and the

In [22]:
# Train it another 20 epochs... how does it do?
train_ptb(model_rnn, train_data, seq_len=20, n_epochs=20, device=device)

epoch i=0
batch=0/181, avg_acc=0.1140625, avg_loss=6.30726203918457
batch=500/181, avg_acc=0.11842698317307693, avg_loss=6.252452674278846
batch=1000/181, avg_acc=0.1183249080882353, avg_loss=6.256296434589461
batch=1500/181, avg_acc=0.11910721628289474, avg_loss=6.252125308388158
batch=2000/181, avg_acc=0.11949837561881188, avg_loss=6.248743521813119
batch=2500/181, avg_acc=0.11988932291666667, avg_loss=6.2447998046875
batch=3000/181, avg_acc=0.1201029594370861, avg_loss=6.243230675703642
batch=3500/181, avg_acc=0.11972323330965909, avg_loss=6.244648881392045
epoch i=1
batch=0/181, avg_acc=0.121875, avg_loss=6.2736858367919925
batch=500/181, avg_acc=0.1200420673076923, avg_loss=6.218623234675481
batch=1000/181, avg_acc=0.12029335171568628, avg_loss=6.220931946997549
batch=1500/181, avg_acc=0.12076480263157895, avg_loss=6.217360325863487
batch=2000/181, avg_acc=0.12090423886138614, avg_loss=6.214897412592822
batch=2500/181, avg_acc=0.12134796626984128, avg_loss=6.210975089905754
batch=

(np.float64(0.13649713405397962), array([5.9325147], dtype=float32))

In [23]:
# RNN — inference, after i=26 epochs of training
for k in range(num_samples):
  y = model_rnn.generate(x, max_new_tokens, temperature=temperature, corpus=corpus)
  # idxs = y.astype("int32")
  print('\n---------------')

there is a pennsylvania tree peabody to '80s for its <unk> sun <eos> the u.s. wall a individuals strategies <eos> that <unk> is <unk> coming and ford for six N <eos> it is n't be service richard be <unk> <unk> with the big area said the nation 's <unk> of the york of the same quarter of a <unk> was <unk> includes the deal of either used of the straight between a <unk> in the loss 's $ N to N N to N to N N to N N by a central house via n't was a los to the work of a new u.s. co. and life compares that <unk> protection in $ N and an <unk> work at the closing <unk> told the american effort inc insurance <eos> <eos> even <unk> are n't share of the ual exchange <eos> the two <unk> of these N billion in the <unk> of one forest <eos> the tax campaign <eos> you just for the few the wait of the settlement ended getting his <unk> <eos> it would want in chicago <unk> the new first products <eos> the <unk> in her more have been original food bills of certain <unk> and not a fourth quarter <eos> th